In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
from sklearn.decomposition import PCA

!pip install matplotlib --quiet
import matplotlib.pyplot as plt
%matplotlib inline

!pip install plotly --quiet
import plotly.express as px

In [ ]:
notetypes = ['eMAR-Medication Administration Note',
 'General Note',
 'Pharmacy Review',
 'COVID-19 Screening note',
 'SBAR - Change of Condition',
 'Physician Note',
 'Behavior Charting',
 'Admission Note',
 'eMar - Shift Level Administration Note',
 'Weight Note',
 'Alert Note',
 'Care Plan Note',
 'Care Management Strategy',
 'Weekly Care Management Meeting']

In [ ]:
processed_path = Path('/data/processed')
processed_path.mkdir(parents=True, exist_ok=True)



In [ ]:
for notetype in notetypes:
    final = pd.read_parquet(processed_path/f'02a-result_{notetype}.parquet')
    output_hosp_cols = [col for col in final.columns if 'hosp_target_3_day_hosp' in col and '_target_' in col]
    final[output_hosp_cols] = final[output_hosp_cols].fillna(False)

    notes_col = [ col for col in final.columns if 'scispacy_' in col]
    final = final.dropna(subset=notes_col)
    print('notes_col length--> ',len(notes_col))

    embeddings_col = [ col for col in final.columns if 'scispacy_' in col and '_ewm' not in col and '_unit_' not in col]
    print('embeddings_col length--> ',len(embeddings_col))

    ewm_col = [ col for col in final.columns if '_ewm' in col]
    print('ewm_col length--> ',len(ewm_col))

    unit_col = [ col for col in final.columns if '_unit_' in col]
    print('unit_col length--> ',len(ewm_col))
    df = final[embeddings_col+output_hosp_cols]
    print('df.shape--> ',df.shape)
    print('===sampling===')
    df_sample = df.sample(n=10000, random_state=1)
    print('df shape after sampling--> ',df_sample.shape)
    print('====sampled hosp_target_3_day_hosp value_counts====')
    print(df_sample['hosp_target_3_day_hosp'].value_counts())
    X = df_sample.drop('hosp_target_3_day_hosp',axis=1)
    Y = df_sample['hosp_target_3_day_hosp']
    # scaled_data = scalar.fit_transform(X)
    pca = PCA(n_components = 3)
    x_pca = pca.fit_transform(X)
    x_pca = pd.DataFrame(x_pca)
    x_pca.columns = ['x_pca_'+str(col) for col in x_pca.columns]
    fig = px.scatter_3d(x=x_pca['x_pca_0'], y=x_pca['x_pca_1'], z=x_pca['x_pca_2'],
              color=Y,title=notetype)
    fig.show()

In [ ]:
# !pip install seaborn --quiet
# import matplotlib.pyplot as plt
# import pandas as pd
# import seaborn as sns
# from mpl_toolkits.mplot3d import Axes3D

# sns.set(style = "darkgrid")

# fig = plt.figure()
# ax = fig.add_subplot(111, projection = '3d')

# x = x_pca['x_pca_0']
# y = x_pca['x_pca_1']
# z = x_pca['x_pca_2']

# ax.set_xlabel("x_pca_0")
# ax.set_ylabel("x_pca_1")
# ax.set_zlabel("x_pca_2")

# ax.scatter(x, y, z, c=Y)

# plt.show()

In [ ]:
# # giving a larger plot
# plt.figure(figsize =(8, 6))
  
# plt.scatter(x_pca['x_pca_0'], x_pca['x_pca_1'], c = Y)
  
# # labeling x and y axes
# plt.xlabel('First Principal Component')
# plt.ylabel('Second Principal Component')